In [1]:
import numpy as np
import pandas as pd

import h5py
import os
import time

from time import perf_counter as pc


os.chdir("C:\\Users\\Vikto\\Documents\\ERNN")
file = h5py.File('DE30_EUR_M1.h5', 'r')
os.chdir("C:\\Users\\Vikto\\Documents\\ERNN")
dset = file[list(file.keys())[0]]
dset.keys()

<KeysViewHDF5 ['axis0', 'axis1', 'block0_items', 'block0_values', 'block1_items', 'block1_values', 'block2_items', 'block2_values']>

In [2]:
data = dset['block0_values'][:]
data = np.column_stack((data, dset['block1_values'][:, 0]))
data = np.column_stack((data, dset['block2_values'][:, 0]))

columns = []
for el in dset['axis0']:
    if el == b'ask_close':
        columns.append('ask_close')
    elif el == b'ask_open':
        columns.append('ask_open')
    elif el == b'ask_high':
        columns.append('ask_high')
    elif el == b'ask_low':
        columns.append('ask_low')
    elif el == b'bid_close':
        columns.append('bid_close')
    elif el == b'bid_open':
        columns.append('bid_open')
    elif el == b'bid_high':
        columns.append('bid_high')
    elif el == b'bid_low':
        columns.append('bid_low')
    elif el == b'volume':
        columns.append('volume')
    elif el == b'time':
        columns.append('time')

df = pd.DataFrame(data=data, columns=columns)
del(data); del(columns)

df['time'] = df['time']/ 10 ** 9

In [3]:
time_start = time.mktime((2005, 1, 1, 0, 0, 0, 0, 0, 0))
time_stop  = time.mktime((2019, 1, 1, 0, 0, 0, 0, 0, 0))

global_data = df[(time_start < df['time']) & (df['time'] < time_stop)].reset_index(drop=True)

global_data.head(11)

,ask_close,ask_high,ask_low,ask_open,bid_close,bid_high,bid_low,bid_open,volume,time
0,4261.200195,4265.200195,4261.200195,4264.200195,4261.200195,4265.200195,4261.200195,4264.200195,43.0,1.104739e+09
1,4257.200195,4261.700195,4256.799805,4261.700195,4257.200195,4261.700195,4256.799805,4261.700195,39.0,1.104739e+09
2,4259.200195,4259.700195,4255.799805,4255.799805,4259.200195,4259.700195,4255.799805,4255.799805,26.0,1.104739e+09
3,4259.200195,4261.200195,4258.700195,4259.700195,4259.200195,4261.200195,4258.700195,4259.700195,26.0,1.104739e+09
4,4259.700195,4260.200195,4258.700195,4259.700195,4259.700195,4260.200195,4258.700195,4259.700195,15.0,1.104739e+09
5,4258.200195,4260.200195,4255.799805,4259.200195,4258.200195,4260.200195,4255.799805,4259.200195,29.0,1.104740e+09
6,4253.799805,4257.700195,4253.799805,4257.700195,4253.799805,4257.700195,4253.799805,4257.700195,25.0,1.104740e+09
7,4254.799805,4254.799805,4252.799805,4253.299805,4254.799805,4254.799805,4252.799805,4253.299805,17.0,1.104740e+09
8,4258.200195,4258.200195,4254.299805,4254.299805,4258.200195,4258.200195,4254.299805,4254.299805,16.0,1.104740e+09
9,4258.700195,4260.200195,4256.799805,4257.700195,4258.700195,4260.200195,4256.799805,4257.700195,22.0,1.104740e+09


In [4]:
time_convert = 480

data = global_data[: -1 - time_convert - len(global_data)%time_convert : time_convert].copy() 
data.loc[:, 'ask_close'] = list(global_data.loc[data.index+time_convert-1,'ask_close'])
data.loc[:, 'bid_close'] = list(global_data.loc[data.index+time_convert-1,'bid_close'])
data.loc[:, 'volume'] = [global_data.loc[k:k+time_convert-1,'volume'].sum() for k in data.index]
data.loc[:, 'ask_high'] = [global_data.loc[k:k+time_convert-1,'ask_high'].max() for k in data.index]
data.loc[:, 'bid_high'] = [global_data.loc[k:k+time_convert-1,'bid_high'].max() for k in data.index]
data.loc[:, 'ask_low'] = [global_data.loc[k:k+time_convert-1,'ask_low'].min() for k in data.index]
data.loc[:, 'bid_low'] = [global_data.loc[k:k+time_convert-1,'bid_low'].min() for k in data.index]

data = data.reset_index(drop=True)

In [5]:
def prediction(data1, slr, hidden_layer, quan_train_data1, types='bid_close'):    
    
    np.random.seed(17)
    
    minimum = np.min(data1[types])
    maximum = np.max(data1[types])
    
    for name in data1.columns[:-1]:
        data1.loc[:, name] = (data1[name] - np.min(data1[name])) / (np.max(data1[name]) - np.min(data1[name]))
        
    columns = ['ask_close', 'ask_high', 'ask_low', 'ask_open', 'bid_close', 'bid_high', 'bid_low', 'bid_open', 'volume']
    data1 = data1.loc[:, columns]
    
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))


    def Derivative_Sigmoid(x):
        return sigmoid(x) * (1 - sigmoid(x))

    
    def training(data1, hidden_layer):

        n, m = data1.shape
#         print(m)
#         m -= 1

        W = np.random.normal(loc=0, scale=1, size=(m, hidden_layer))
        V = np.random.normal(loc=0, scale=1, size=hidden_layer)
        C = np.random.normal(loc=0, scale=1, size=(hidden_layer, hidden_layer))

        def step(data1_row, current_layer):

            input_hidden = np.matmul(data1_row, W) + np.matmul(current_layer, C)
            out_hidden = sigmoid(input_hidden)
            predict = np.dot(V, out_hidden)

            return predict, out_hidden, input_hidden


        for nit in range(n-1):
            if nit:
                predict, out_current, input_hidden = step(data1.loc[nit], out_current)    
            else:
                predict, out_current, input_hidden = step(data1.loc[nit], np.zeros(hidden_layer))

            error = data1.loc[nit, types] - predict

            V += slr * error * out_current
            W += slr * error * np.outer(V * Derivative_Sigmoid(input_hidden), data1.loc[nit]).T
            C += slr * error * np.outer(V * Derivative_Sigmoid(input_hidden), out_current).T

        return W, V, C, out_current
    
    
    W, V, C, out_current = training(data1, hidden_layer)
    predict = np.dot(V, sigmoid(np.matmul(data1.loc[n-1], W) + np.matmul(out_current, C)))
    
    predict = predict * (maximum - minimum) + minimum
    return predict
                     

In [6]:
n, m = data.shape
m -= 1
hidden_layer = 30
quan_train_data = data.shape[0] - 1
slr = 0.1
types = 'bid_close'
data1 = data.copy(deep=True)

predict = prediction(data1, slr, hidden_layer, quan_train_data, types)

error = data.loc[quan_train_data, types] - predict
error

-1218.2621861336884

In [7]:
predict/data.loc[quan_train_data, types] * 100 - 100

11.529695434093526

In [8]:
-775.3149158149645
780.1949154806316

780.1949154806316